In [ ]:
import my_mps
import d_dmrg
from bosehubbardfile import BoseHubbardModel
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

#code adapted from tutorial 9

In [ ]:
def my_expectation_value(psi, model):
    for i in range(model.L):
        ket = psi[i]
        bra = np.conj(psi[i])
        contraction = np.tensordot(ket, model.fixed_H_mpo[i], (1,2)) #vL [i] vR , hL hR [hup] hdown
        nextcontraction = np.tensordot(contraction, bra, axes=([4], [1])) #vL vR hL hR [hdown] , vL [i] vR
        #firstcontraction.append(nextcontraction)
        if i == 0: 
            energy = nextcontraction
        else: 
            #print("Energy", energy.shape)
            #print("Contraction", nextcontraction.shape)
            energy = np.tensordot(energy, nextcontraction, axes=([1,3,5],[0,2,4])) #vL [vR] hL [hR] vL [vR],  [vL] vR [hL] hR [vL] vR
            energy = np.transpose(energy, (0,3,2,4,1,5))
    return np.squeeze(energy)

In [ ]:
import copy
def dmrg_gs(L, t, U, mu, nc, max_N_sweeps=10, chi_max=50, eps=1.e-10):
    model = BoseHubbardModel(L, t, U, mu, nc)
    psi = my_mps.init_spinup_MPS(L, nc)
    psi_initial = copy.deepcopy(psi.Bs)
    eng = d_dmrg.DMRGEngine(psi, model, chi_max=chi_max, eps=eps)
    Eprev = np.inf
    energyss = []
    for i in range(max_N_sweeps):
        print("sweep", i)
        eng.sweep()
        E = my_expectation_value(psi.Bs, model)
        #if abs((E - Eprev)) < eps: 
        #    print('')  # newline
        #    break
        #print(E)
        energyss.append(E)
        #Eprev = E
    print("max final bond dimension: ", max(psi.get_chi()))
    return model, psi, energyss

In [ ]:
plt.figure()
params = [(3, 1.5), (5, 2.0), (7, 2.5), 
          (9, 1.0)]
for L, t in params:
    print("L =", L,  "t =", t)
    model = BoseHubbardModel(L, t, 1.0, 1.0, 2)
    psi = my_mps.init_spinup_MPS(L, 2)
    eng = d_dmrg.DMRGEngine(psi, model, chi_max=130, eps=1.e-13)
    Es = []
    for i in range(10):
        Es.append(model.energy(psi))
        eng.sweep()
    Es = np.array(Es)
    lbl = '$L={L:d}, t={t}$'.format(L=L, t=t)
    plt.plot(Es, label=lbl)#, linestyle=linestyle
    print("max chi = ", max(psi.get_chi()))
plt.ylabel('Energy')
plt.xlabel('sweep')
plt.legend(loc='upper right')
plt.show()

In [ ]:
#Correlation function section 

def correlation_function(psi, X, Y, i):
    L = psi.L
    assert i < L
    corrs = []
    # first case: j = i
    # Note: psi.get_theta(i) = Lambda[i] B[i] with axes vL i vR
    theta = psi.get_theta1(i)  # vL i vR
    X_Y_theta = np.tensordot(np.dot(X, Y), # i [i*], [i] i*
                             theta, axes=([1], [1]))  # i [i*], vL [i] vR
    contr = np.tensordot(theta.conj(), X_Y_theta, axes=([0, 1, 2], [1, 0, 2]))  # [vL*] [i*] [vR*], [i] [vL] [vR]
    corrs.append(contr)
    # now for i < j
    theta = psi.get_theta1(i)  # vL i vR
    LP = np.tensordot(X, theta, axes=([1], [1]))  # i [i*], vL [i] vR
    LP = np.tensordot(theta.conj(), LP, axes=([0, 1], [1, 0])) # [vL*] [i*] vR*, [i] [vL] vR
    # now LP has axes vR*, vR
    for j in range(i+1, L):
        # contract with B Y B* to evaluate the correlation function for given j
        B = psi.Bs[j]  # vL j vR
        Y_B = np.tensordot(Y, B, axes=([1], [1])) # j [j*], vL [j] vR
        contr = np.tensordot(LP, Y_B, axes=([1], [1])) # vR* [vR], j [vL] vR
        contr = np.tensordot(contr, B.conj(), axes=([0, 1, 2], [0, 1, 2])) # [vR*] [j] [vR], [vL*] [j*] [vR*]
        corrs.append(contr.item())
        # get next LP for next i
        LP = np.tensordot(LP, B, axes=([1], [0])) # vR* [vR], [vL] j vR
        LP = np.tensordot(B.conj(), LP, axes=([0, 1], [0, 1])) # [vL*] [j*] vR*, [vR*] [j] vR
    return np.real_if_close(corrs)


In [ ]:
mus = np.arange(-0.1, 2.0, 0.1)
ts = np.arange(0.0, 0.3, 0.01)
Us = [1]
ncs = [0,1,2,3,4]
corrs = []
densities = []
t_U = []
mu_U = []
for U in Us:
    for mu in mus:
        print('-'*40)
        for t in ts:
            print("U =", U)
            print("mu =", mu)
            print("t =", t)
            t_U.append(t/U)
            mu_U.append(mu/U)
            model, psi, energyss = dmrg_gs(8, t, U, mu, 2, chi_max=150, eps=1.e-10)
            cr = model.init_creation_op() 
            ann = model.init_annihilation_op()
            num = model.init_number_op()
            id = np.eye(model.d)
            corr = correlation_function(psi, cr, ann, model.L//4)
            print(corr)
            corrs.append(corr[(3*7)//4]) #changes for different system sizes
            den_exp_val = psi.site_expectation_value(num)  # only for i >= L//4
            print("<num> =", den_exp_val)
            den_exp_val_avg = np.average(den_exp_val)
            #density = correlation_function(psi, num, id, model.L//2)
            #print(density)
            densities.append(den_exp_val_avg)
    t_U = np.array(t_U)
    mu_U = np.array(mu_U)
    ncs = np.array(ncs)
    print(t_U)
    print(mu_U)
    corrs = np.reshape(corrs, (len(np.unique(mu_U)),len(np.unique(t_U))))
    densities = np.reshape(densities, (len(np.unique(mu_U)),len(np.unique(t_U)))) 
    print("max chi = ", max(psi.get_chi()))
print(corrs.shape)
print(densities.shape)


In [ ]:
plt.figure()
mu_U_lbl = np.around(np.unique(mu_U), decimals = 2)
t_U_lbl = np.around(t_U, decimals=2)
plt.xticks(ticks=np.arange(len(t_U)),labels=t_U_lbl,rotation=90)
plt.yticks(ticks=np.arange(len(mu_U)),labels=mu_U_lbl)
hm2=plt.imshow(densities, cmap='Blues',interpolation="nearest")
plt.colorbar(hm2)
plt.xlabel("t/U")
plt.ylabel("mu/U")
plt.show()

In [ ]:
plt.figure()
mu_U_lbl = np.around(np.unique(mu_U), decimals = 2)
t_U_lbl = np.around(t_U, decimals=2)
plt.xticks(ticks=np.arange(len(t_U)),labels=t_U_lbl,rotation=90)
plt.yticks(ticks=np.arange(len(mu_U)),labels=mu_U_lbl)
hm=plt.imshow(corrs, cmap='viridis',interpolation="nearest")
plt.colorbar(hm)
plt.show()